In [1]:
from working_with_files import *
from preprocessing import *
import pandas as pd
from datetime import datetime
from collections import Counter
import gensim
from string import punctuation
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

In [49]:
def sort_json_by_official_dates(corpus):
    corpus1 = [] # < 2008
    corpus2 = [] # 2008 - 2013
    corpus3 = [] # 2013 - 2016
    corpus4 = [] # > 2016
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for text_data in corpus:
        if text_data['id'] not in [8079, 10995, 11494, 11848, 11854, 11858, 26987, 30920]:
            try:
                date = datetime.strptime(text_data['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(text_data['date'], '%m-%Y').date()
            if date <= date1:
                corpus1.append(text_data)
            if date > date1 and date <= date2:
                corpus2.append(text_data)
            if date > date2 and date <= date3:
                corpus3.append(text_data)
            if date > date3:
                corpus4.append(text_data)
    return corpus1, corpus2, corpus3, corpus4

# tokenize text into list of lists for building w2v model
def tokenize_text(lemmas):
    stopwords = set(get_lexicon_from_file('ru_stopwords_extended.txt'))
    text = ' '.join(lemmas)
    sentences = sent_tokenize(text)
    sentences_tokenized = []
    for sentence in sentences:
        words = [token for token in word_tokenize(sentence) if token.isalpha() and token.isascii() == False 
                 and len(token) > 2 and token not in stopwords]
        if words != []:
            sentences_tokenized.append(words)
    return sentences_tokenized

# build vocabulary and train word2vec model
def build_w2v_model(corpus, min_count):
    documents = []
    for text_data in corpus:
        documents.extend(tokenize_text(text_data['lemmas']))
    w2v_model = gensim.models.Word2Vec(documents, size=300, window=5, min_count=min_count)
    return w2v_model

# count number of texts and symbols in a corpus
def get_json_stat(corpora):
    n = 0
    for corpus in corpora:
        num_words = 0
        for text_data in corpus:
            for pos_tag in text_data['pos_tags']:
                if pos_tag != 'NO_POS':
                    num_words += 1
        print('Corpus {}: {} texts, {} words'.format(n, len(corpus), num_words))
        n += 1

# get n words most similar to the input word        
def get_most_similar(word, topn, models):
    print(word.upper(), '\n')
    titles = ['> 2008', '2008 - 2013', '2013 - 2016', '> 2016']
    indx = 0
    for model in models:
        title = titles[indx]
        try:
            print(title, '\n', model.wv.most_similar(positive=word, topn=topn), '\n')
        except KeyError:
            print(title, '\nNo such word in vocabulary\n')
        indx += 1 
        
def sort_json_for_semantic_shifts(corpus):
    corpus1 = [] # < 2008
    corpus2 = [] # < 2013
    corpus3 = [] # < 2016
    # corpus4 = corpus (all data)
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for text_data in corpus:
        if text_data['id'] not in [8079, 10995, 11494, 11848, 11854, 11858, 26987, 30920]:
            try:
                date = datetime.strptime(text_data['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(text_data['date'], '%m-%Y').date()
            if date <= date1:
                corpus1.append(text_data)
            if date <= date2:
                corpus2.append(text_data)
            if date <= date3:
                corpus3.append(text_data)
    return corpus1, corpus2, corpus3, corpus

def get_most_freq_ngrams(corpus, n, top=None, pos='any', print_ngrams=True):
    all_ngrams = []
    for text_data in corpus:
        stopwords = get_lexicon_from_file('ru_stopwords_lemmatized.txt')
        stopwords.extend(list('абвгдеёжзийклмнопрстуфхцчшщ'))
        punct_n_stopwords = list(punctuation) + stopwords
        if n == 1:
            if pos == 'any':
                all_ngrams.extend([lemma.lower() for lemma in text_data['lemmas'] 
                               if lemma not in stopwords and text_data['pos_tags'][text_data['lemmas'].index(lemma)] != 'NO_POS'])
            else:
                all_ngrams.extend([lemma.lower() for lemma in text_data['lemmas'] 
                               if lemma not in stopwords and text_data['pos_tags'][text_data['lemmas'].index(lemma)] == pos])
        elif n == 2:
            text = [token for token in text_data['lemmas'] if token.isalpha() and token not in stopwords]
            if text != []:
                bigrams = list(nltk.bigrams(text))
                all_ngrams.extend(bigrams)
        else:
            print(n)
            return 'N should be either 1 or 2!'
    
    freq_dict = sorted(dict(Counter(all_ngrams)).items(), key=lambda kv: kv[1], reverse=True)
    if print_ngrams != False:
        for i in range(top):
            print(freq_dict[i])
            
        
    #print(freq_dict[:top])
    return freq_dict  

# count number of texts and words in a corpus
def get_json_stat(corpora, print_stat=True):
    n = 0
    for corpus in corpora:
        num_words = 0
        for text_data in corpus:
            for pos_tag in text_data['pos_tags']:
                if pos_tag != 'NO_POS':
                    num_words += 1
        if print_stat == True:
            print('Corpus {}: {} texts, {} words'.format(n, len(corpus), num_words))
        n += 1
        return len(corpus), num_words

In [25]:
corpus = get_json_from_file('mfa_texts_rhetoric.json')
# split corpus into 4 subcorpora
corpora = sort_json_by_official_dates(corpus)

In [52]:
def get_freq_dynamics(corpora, n, pos='any'):
    
    # get ngram frequency dicts for every corpus & save corpora lengths to later normalize frequencies
    freq_dicts = []
    for corpus in corpora:
        corpus_data = {}
        corpus_data['freq_dict'] = dict(get_most_freq_ngrams(corpus, n, pos=pos, print_ngrams=False))
        corpus_data['corpus_size'] = get_json_stat([corpus], print_stat=False)[1]
        freq_dicts.append(corpus_data)
        
    # get all ngrams in freq dicts
    all_ngrams = []
    for freq_dict in freq_dicts:
        for ngram in freq_dict['freq_dict'].keys():
            if ngram not in all_ngrams:
                all_ngrams.append(ngram)
                
    # find only ngrams present in every corpus
    common_ngrams = []
    for ngram in all_ngrams:
        num_entries = 0
        for freq_dict in freq_dicts:
            if ngram in freq_dict['freq_dict'].keys():
                num_entries += 1
        if num_entries == len(freq_dicts):
            common_ngrams.append(ngram)
   
    # get ngrams (relative) frequency  
    ngrams_freqs = {}
    for ngram in common_ngrams:
        ngram_freqs = []
        for freq_dict in freq_dicts:
            # normalize ngram frequencies 
            ngram_freqs.append(freq_dict['freq_dict'][ngram] / freq_dict['corpus_size'])
        ngrams_freqs[ngram] = ngram_freqs
    
    # get ngrams freq dynamics
    ngrams_freqs_dynamics = {}
    for ngram, freqs in ngrams_freqs.items():
        growth_rates = []
        for i in range(1, len(freqs)):
            growth_rates.append(freqs[i] / freqs[i-1])
        avg_growth = sum(growth_rates) / len(growth_rates)
        ngrams_freqs_dynamics[ngram] = avg_growth
    
    # get freq growth rate
    growth_rate = sorted(ngrams_freqs_dynamics.items(), key=lambda kv: kv[1], reverse=True)
    print(growth_rate)
    return growth_rate

In [51]:
growth_rate = get_freq_dynamics(corpora, 1)

1712197
2070100
2113340
2288457
[('астанинский', 150.38861513245425), ('иго', 105.82768058031341), ('донбасс', 62.85124634779231), ('сара', 61.28716696975027), ('донецкий', 51.20117945152588), ('усыновление', 37.34969964620725), ('луганский', 31.684811860390795), ('юго-восток', 28.444940534298535), ('мосул', 25.191042317356334), ('полицентричный', 24.654911696679957), ('каддафи', 24.21425953756913), ('децентрализация', 22.619760296832723), ('пират', 22.59504325821786), ('мали', 22.113669234335266), ('озхо', 21.850674642393955), ('тегеранский', 21.23611249488521), ('клинтон', 21.078876200767198), ('модернизационный', 20.76064550519628), ('саркозить', 20.431769635824022), ('ан', 20.345158950341105), ('татарин', 19.18888489922112), ('новичок', 19.046758439516505), ('киевский', 17.37278303846125), ('абэ', 17.051859612839234), ('тишина', 15.851053726955406), ('расмуссен', 15.709880775906093), ('халифат', 15.34368641114718), ('крым', 14.108611901531761), ('донецк', 14.072324478772124), ('нер

In [53]:
growth_rate_adj = get_freq_dynamics(corpora, 1, pos='A')

[('астанинский', 150.38861513245425), ('донецкий', 51.20117945152588), ('луганский', 31.684811860390795), ('полицентричный', 24.654911696679957), ('тегеранский', 21.23611249488521), ('модернизационный', 20.76064550519628), ('киевский', 17.37278303846125), ('нервно-паралитический', 13.838735869703221), ('ливийский', 12.201897818013578), ('инклюзивный', 11.44679221610493), ('токсичный', 10.759952043257556), ('русофобский', 10.1132004625546), ('восточноазиатский', 9.514787628855407), ('медийный', 8.161248022564264), ('подконтрольный', 7.612508103024434), ('вашингтонский', 7.47987012359657), ('умеренный', 7.442741360579014), ('елисейский', 7.374388932006774), ('пиратский', 6.956574073375144), ('неназванный', 6.884487199262325), ('келейный', 6.859297262592455), ('нигерийский', 6.747075983101776), ('докризисный', 6.7287848457109325), ('ярославский', 5.974767980060098), ('приемный', 5.9557513877829535), ('косовоалбанский', 5.940348954199282), ('ирландский', 5.93222177130574), ('женевский', 5.

In [54]:
growth_rate_verb = get_freq_dynamics(corpora, 1, pos='V')

[('саркозить', 20.431769635824022), ('стрелять', 13.434184671613005), ('усыновлять', 10.921895952542938), ('пожаловать', 9.6881031840944), ('изгонять', 9.178822071956587), ('трогать', 7.34642906297631), ('казнить', 6.946702090295929), ('отвоевывать', 6.374004279483185), ('заручаться', 6.346908834539693), ('нападывать', 6.209030394350752), ('атаковать', 5.971692309942321), ('отфиксировать', 5.648643951036219), ('извиняться', 5.278099584916112), ('чествовать', 5.24551069924235), ('срежиссировать', 5.136026025845861), ('приговаривать', 5.0318006052192565), ('затевать', 4.849228580362577), ('фальсифицировать', 4.401791007289252), ('обжаловать', 4.401218730938222), ('проливать', 4.376259194058688), ('волноваться', 4.336147723885595), ('подминать', 4.225662783582593), ('переподтверждать', 4.190415143320837), ('раздавать', 4.164285863125366), ('ослабевать', 4.151057857827539), ('заказывать', 4.147554517140081), ('отравлять', 4.124317543929843), ('аккредитоваться', 4.100748642130145), ('демили

In [55]:
growth_rate_adv = get_freq_dynamics(corpora, 1, pos='ADV')

[('давным-давно', 6.795239919533581), ('страшно', 6.213690224141518), ('моментально', 4.259126207678849), ('однажды', 4.036037740329918), ('армяно', 3.935851571505397), ('прилично', 3.923315977149164), ('глобально', 3.235216422645854), ('исправно', 3.207868405139468), ('методично', 3.133128099741397), ('слышно', 3.003242875417966), ('по-честному', 2.9609401162170603), ('националистически', 2.957071286224048), ('преднамеренно', 2.8897157524116577), ('неважно', 2.81236984089807), ('бессмысленно', 2.8013496847967656), ('высокомерно', 2.775685383144996), ('безосновательно', 2.7684633457438372), ('блестяще', 2.7235696509924807), ('зеркально', 2.703979853257033), ('заочно', 2.6965208445044837), ('убито', 2.692533909720414), ('следом', 2.691763822515657), ('нечестно', 2.6692215365743697), ('неприлично', 2.645216503729115), ('настороженно', 2.635890754418354), ('замечательно', 2.5937416794367922), ('вслух', 2.5711014658436557), ('лицемерно', 2.5239207012569795), ('осознанно', 2.462496714082840

In [56]:
growth_rate_intj = get_freq_dynamics(corpora, 1, pos='INTJ')

[('увы', 0.8858685029939867), ('ой', 0.794778261660028)]


In [57]:
growth_rate_part = get_freq_dynamics(corpora, 1, pos='PART')

[('ан', 5.387300115917076), ('еще', 2.5276591838411187), ('де', 2.4069439679109466), ('ван', 2.1224990632395806), ('якобы', 2.0224032034241195), ('неужели', 1.8058142739603975), ('себе', 1.670263545480157), ('ладно', 1.584483832502685), ('угодно', 1.5813861007409546), ('разве', 1.5549755729028767), ('хоть', 1.5414614054428502), ('пожалуйста', 1.5283727958804068), ('все', 1.462883790693251), ('будто', 1.442433744435343), ('исключительно', 1.4065651179329723), ('хотя', 1.4059023832092004), ('наверняка', 1.3899223248550368), ('все-таки', 1.2962048779596973), ('просто', 1.2892233725290276), ('хорошо', 1.2799998626292768), ('тоже', 1.2783941161472179), ('прямо', 1.251906590588558), ('было', 1.2336876193735022), ('никак', 1.2216107430188217), ('вроде', 1.2205784628515872), ('всего', 1.1961507507299645), ('пускай', 1.1877366101456939), ('се', 1.1099838184637523), ('уж', 1.1059449987848462), ('мол', 1.0794646194031345), ('пусть', 1.040038276284415), ('да', 0.9855960791784558), ('именно', 0.967

In [58]:
growth_rate_advpro = get_freq_dynamics(corpora, 1, pos='ADVPRO') # местоименные наречия

[('вон', 2.5304184519805184), ('как-нибудь', 2.505233580335581), ('некогда', 2.5018407514086243), ('всюду', 2.3021477485853876), ('всего', 2.2398607692569885), ('нисколько', 2.02266620738825), ('где-нибудь', 1.8982748756323655), ('куда-либо', 1.8861886621042006), ('куда-нибудь', 1.8000915170358454), ('откуда', 1.67993452649161), ('почему-то', 1.6151970019264068), ('когда-нибудь', 1.609444735481371), ('зачем', 1.570242432365853), ('туда', 1.5334641677971483), ('откуда-то', 1.5283327739323038), ('куда-то', 1.5178080155782883), ('как-то', 1.459899487562554), ('оттуда', 1.4332138079362722), ('где-либо', 1.3839409757001804), ('никуда', 1.363064748453863), ('почему', 1.3462354699696004), ('где-то', 1.3300151820617956), ('отчего', 1.2526167901677905), ('куда', 1.2217671873909681), ('отовсюду', 1.2178680389625385), ('нигде', 1.1995208548470595), ('никогда', 1.1513172244147436), ('всегда', 1.1145675436241167), ('никак', 1.0579261165920535), ('когда-то', 1.0240099496352062), ('оттого', 1.0224881

In [59]:
growth_rate_anum = get_freq_dynamics(corpora, 1, pos='ANUM') # ЧИСЛИТЕЛЬНОЕ-ПРИЛАГАТЕЛЬНОЕ

[('тысячный', 2.160033293840572), ('восемнадцатый', 2.0034947808769217), ('пятнадцатый', 2.0034947808769217), ('двенадцатый', 1.8970404997837065), ('двадцатый', 1.7594831613033535), ('девятый', 1.729078040063161), ('шестой', 1.6198955449424677), ('тринадцатый', 1.615844684551222), ('седьмой', 1.519465102120232), ('одиннадцатый', 1.3935970156347963), ('восьмой', 1.3513116787692683), ('миллионный', 1.192462848217986), ('пятый', 1.1859584386748019), ('второй', 1.0166947684191483), ('десятый', 0.9888920461331723), ('четвертый', 0.9875529871134082), ('первый', 0.9388589087734376), ('третий', 0.8764679751674794), ('миллиардный', 0.8463254654853983), ('девяностый', 0.7234630425539782)]


In [60]:
growth_rate_apro = get_freq_dynamics(corpora, 1, pos='APRO') # местоимение-прилагательное

[('какой-нибудь', 3.139476674646042), ('такой-то', 1.9667330837685988), ('твой', 1.7733754524083565), ('некий', 1.594599458198341), ('тот-то', 1.5065228629475724), ('немногий', 1.3744181192859937), ('ваш', 1.2985143911335302), ('какой-то', 1.253755271034233), ('остальной', 1.2250864251897962), ('никакой', 1.2240095318548792), ('никой', 1.182286070857404), ('сей', 1.1747916566518353), ('чей', 1.1487970067917506), ('какой', 1.1313510650031515), ('какой-либо', 1.1062991222939078), ('каковой', 1.0915067600687542), ('сам', 1.0571786594762926), ('чей-либо', 1.0524609000583645), ('самый', 1.0490014835476138), ('любой', 1.036020314411395), ('свой', 1.0314309118469729), ('мой', 0.9967651703719692), ('их', 0.9932714161091463), ('многий', 0.9909204119674276), ('его', 0.9886663004216696), ('некоторый', 0.9885770473131591), ('другой', 0.9777381651323086), ('таковой', 0.9729953417628187), ('такой', 0.9693096751341352), ('кой', 0.9649842414175566), ('каждый', 0.9598523199356367), ('иной', 0.910694371

In [61]:
growth_rate_conj = get_freq_dynamics(corpora, 1, pos='CONJ') 

[('ан', 30.245973646616807), ('благо', 2.0682548333480297), ('чтоб', 2.061522121245686), ('аль', 2.0550875559036195), ('плюс', 1.786561095032998), ('притом', 1.751730098988905), ('итак', 1.7174168359741844), ('соответственно', 1.5212329699726588), ('будто', 1.5091077226151552), ('хоть', 1.4944538700690047), ('коль', 1.448543831456315), ('словно', 1.4083926334287715), ('сколько', 1.390757387996687), ('зато', 1.3646428962960753), ('али', 1.338420857933399), ('точно', 1.3373136320844032), ('нежели', 1.2700148776506388), ('либо', 1.2146393478046138), ('иначе', 1.181087033593636), ('пусть', 1.0976305945042337), ('хотя', 1.036623192436107), ('равно', 1.0174905065429838), ('причем', 1.003672775490149), ('однако', 0.9616552939494215), ('дабы', 0.9367618481566433), ('да', 0.916887101383832), ('пока', 0.9069394137408344), ('ведь', 0.886224158807701), ('поскольку', 0.8476530112934358), ('ибо', 0.8455762691190148), ('лишь', 0.8418961607701526), ('впрочем', 0.7783752994310063), ('едва', 0.776180285

In [62]:
growth_rate_com = get_freq_dynamics(corpora, 1, pos='COM') # часть композита - сложного слова

[('би', 4.260446879546097), ('таджикско', 3.3142944458069574), ('германо', 2.539894830741386), ('историко', 2.0488093359752404), ('экономико', 2.0022894474298663), ('водо', 1.9646534504860602), ('электро', 1.7682856885590217), ('санкт', 1.6376122289227961), ('бельгийско', 1.6340830111659166), ('американо', 1.4481701706247019), ('англо', 1.2437721187635964), ('анти', 1.236555159883937), ('информационно', 1.166092276071405), ('политико', 1.0291143810146097), ('авиа', 0.9172204671071281), ('северо', 0.9112103965698681), ('мини', 0.858043379609955), ('российско', 0.8460351100545455), ('юго', 0.6551936971375077), ('грузино', 0.44899993472273464), ('энерго', 0.43128693478824526), ('азово', 0.42114372670777217)]


In [63]:
growth_rate_spro = get_freq_dynamics(corpora, 1, pos='SPRO') # местоимение-сущ

[('некого', 3.319681878880857), ('нечего', 1.8302977828569593), ('немногие', 1.7897011956041595), ('что-нибудь', 1.7436128774622768), ('кто-нибудь', 1.725092680207535), ('некто', 1.3490013611816367), ('что-то', 1.3073590441977618), ('никто', 1.2842045199337984), ('прочее', 1.2745947257372685), ('кое-кто', 1.1870009561261623), ('кто-то', 1.142740565027853), ('кто-либо', 1.1267851173870225), ('что-либо', 1.117563050323852), ('многие', 1.0967021647550335), ('нечто', 1.0935654104841674), ('кое-что', 1.0633023413047278), ('се', 1.0408445750411406), ('все', 1.0183213898451682), ('некоторые', 0.9991525756387758), ('любой', 0.9881157188921179), ('свое', 0.976386053015707), ('оно', 0.9464896465947286), ('многое', 0.7759034300457618), ('немногое', 0.7426593810747987)]
